# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 1


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 935
po2 935
albumin 558
hba1c 444
lymphocyte 585
height 415
urine-ketone 343
crp 57


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,rr,saps2,sbp,scr,sodium,sofa,use_NaHCO3,uti,wbc,weight
0,13616762,20066510,31257435,True,NaN,64,NaN,NaN,NaN,NaN,...,NaN,30,NaN,NaN,NaN,3,0.0,0,NaN,130.0
0,17922352,20070680,39895564,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,26,NaN,NaN,NaN,5,0.0,0,NaN,NaN
0,11236474,20146147,34965843,True,NaN,40,NaN,NaN,NaN,NaN,...,NaN,21,NaN,NaN,NaN,3,0.0,1,NaN,NaN
0,18135181,20173681,30217596,True,NaN,54,NaN,NaN,NaN,NaN,...,NaN,24,NaN,NaN,NaN,1,0.0,0,NaN,103.5
0,12851923,20235113,35081736,False,NaN,84,NaN,NaN,NaN,NaN,...,NaN,42,NaN,NaN,NaN,5,0.0,1,NaN,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,11084536,29816115,31328473,False,NaN,25,NaN,NaN,NaN,NaN,...,NaN,17,NaN,NaN,NaN,1,0.0,0,NaN,NaN
0,11166681,29877040,38192611,True,NaN,69,NaN,NaN,NaN,NaN,...,NaN,29,NaN,NaN,NaN,4,0.0,0,NaN,NaN
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,NaN,32,NaN,NaN,NaN,3,0.0,0,NaN,47.4
0,18624988,29915993,31559071,True,NaN,82,NaN,NaN,NaN,NaN,...,NaN,63,NaN,NaN,NaN,11,0.0,0,NaN,120.3


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

47244734

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

45620088

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-20 14:13:26.951254: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-20 14:13:30.802038: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

2024-06-20 14:13:34.327863: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 14:13:34.365741: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an 

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 1:03 3s/step - AUC: 0.3824 - accuracy: 0.4062 - loss: 0.7143 - precision: 0.4545 - recall: 0.5882

 7/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - AUC: 0.4944 - accuracy: 0.4973 - loss: 0.6992 - precision: 0.5079 - recall: 0.6523

13/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.5308 - accuracy: 0.5266 - loss: 0.6917 - precision: 0.5378 - recall: 0.6377 

...

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.9685 - accuracy: 0.9345 - loss: 0.2195 - precision: 0.9384 - recall: 0.9311 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.9686 - accuracy: 0.9344 - loss: 0.2190 - precision: 0.9387 - recall: 0.9306 - val_AUC: 0.7086 - val_accuracy: 0.6495 - val_loss: 1.0430 - val_precision: 0.6400 - val_recall: 0.6667


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - AUC: 0.5675 - accuracy: 0.5000 - loss: 0.7937 - precision: 0.5625 - recall: 0.5000

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.6985 - accuracy: 0.6642 - loss: 0.6603 - precision: 0.7036 - recall: 0.6307 


Loses: [0.5630605220794678, 0.5791968107223511, 0.5683882236480713, 0.6177935004234314, 0.636925458908081] 0.5930729031562805 0.029109369408378563
AUCs: [0.7920339107513428, 0.7628953456878662, 0.8005739450454712, 0.7240437269210815, 0.7228612899780273] 0.7604816436767579 0.03271628626202944
Accuracies: [0.6983470916748047, 0.71074378490448, 0.7272727489471436, 0.6528925895690918, 0.6763485670089722] 0.6931209564208984 0.02609159271626927
Precisions: [0.75, 0.7190082669258118, 0.7567567825317383, 0.682692289352417, 0.699999988079071] 0.7216914653778076 0.02838874802295941
Recals: [0.6097561120986938, 0.707317054271698, 0.6829268336296082, 0.5819672346115112, 0.631147563457489] 0.6426229596138 0.04626177580328277


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - AUC: 0.4444 - accuracy: 0.4375 - loss: 0.7382 - precision: 0.5000 - recall: 0.1111

 7/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.5194 - accuracy: 0.5098 - loss: 0.7058 - precision: 0.5562 - recall: 0.1656

13/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.5395 - accuracy: 0.5234 - loss: 0.6988 - precision: 0.5487 - recall: 0.1925

19/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.5563 - accuracy: 0.5283 - loss: 0.6944 - precision: 0.5561 - recall: 0.2119

...

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.9793 - accuracy: 0.9301 - loss: 0.1876 - precision: 0.9696 - recall: 0.8932

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.9793 - accuracy: 0.9302 - loss: 0.1874 - precision: 0.9692 - recall: 0.8938 - val_AUC: 0.7267 - val_accuracy: 0.6528 - val_loss: 1.0622 - val_precision: 0.7123 - val_recall: 0.5306


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - AUC: 0.5873 - accuracy: 0.4688 - loss: 0.7380 - precision: 0.5385 - recall: 0.3889

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.7052 - accuracy: 0.6104 - loss: 0.6434 - precision: 0.6717 - recall: 0.5132 


Loses: [0.5740228295326233, 0.5947946310043335, 0.5560059547424316, 0.6233808398246765, 0.6252041459083557] 0.5946816802024841 0.0271214869965892
AUCs: [0.7848944664001465, 0.741716206073761, 0.8190544843673706, 0.7259221076965332, 0.7236189842224121] 0.7590412497520447 0.03720685083219816
Accuracies: [0.6983470916748047, 0.6900826692581177, 0.7479338645935059, 0.6570248007774353, 0.6307054162025452] 0.6848187685012818 0.03973663063883796
Precisions: [0.6838235259056091, 0.6967213153839111, 0.7279411554336548, 0.6535432934761047, 0.6701030731201172] 0.6864264726638794 0.02523483643732314
Recals: [0.7560975551605225, 0.6910569071769714, 0.8048780560493469, 0.6803278923034668, 0.5327869057655334] 0.6930294632911682 0.0920115101925426
